In [17]:
from datetime import datetime
import json
import logging
from pathlib import Path
import re
import time
from typing import List, Tuple
from tempfile import NamedTemporaryFile

import dask.array as da
from dask.diagnostics import ProgressBar
from dask.distributed import Client, as_completed
import dask_geopandas as dgd 
import hydra
import geopandas as gpd
import numpy as np
from omegaconf import DictConfig, OmegaConf
import pandas as pd
from pyproj import CRS
from tqdm.notebook import tqdm
import xarray as xr
import zarr

log = logging.getLogger(__name__)

In [18]:
client = Client(dashboard_address=':8989')
client

/home/tkb5476/.conda/envs/marquette_env/lib/python3.11/site-packages/distributed/node.py:182: UserWarning: Port 8989 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 37403 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:37403/status,
Dashboard: http://127.0.0.1:37403/status,Workers: 8
Total threads: 32,Total memory: 251.53 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:38479,Workers: 8
Dashboard: http://127.0.0.1:37403/status,Total threads: 32
Started: Just now,Total memory: 251.53 GiB
Comm: tcp://127.0.0.1:40535,Total threads: 4
Dashboard: http://127.0.0.1:38591/status,Memory: 31.44 GiB
Nanny: tcp://127.0.0.1:42333,


In [31]:
json_data = '''
{
  "name": "MERIT",
  "data_path": "/data/tkb5476/projects/marquette/data/",
  "dx": 2000,
  "buffer": 0.3334,
  "units": "mm/day",
  "date_codes": "${data_path}/date_codes.json",
  "crs": {
    "wgs": "epsg:4326",
    "utm18": "epsg:32618"
  },
  "is_streamflow_split": true,
  "start_date": "01-01-1980",
  "end_date": "12-31-2019",
  "num_cores": 20,
  "continent": 7,
  "area": 3,
  "zone": "${continent}${area}",
  "save_paths": {
    "attributes": "${data_path}/${name}/streamflow/attributes.csv",
    "flow_lines": "${data_path}/${name}/raw/flowlines",
    "basins": "${data_path}/${name}/raw/basins/cat_pfaf_${zone}_MERIT_Hydro_v07_Basins_v01_bugfix1.shp",
    "huc10": "${data_path}/HUC/huc_10_CONUS.shp",
    "streamflow_files": "${data_path}/${name}/streamflow/dpl_v2/dHBV"
  },
  "zarr": {
    "edges": "${data_path}/${name}/zarr/dpl_v2/${zone}_edges/",
    "sorted_edges_keys": "${data_path}/${name}/zarr/dpl_v2/${zone}_edge_keys/",
    "HUC_TM": "${data_path}/${name}/zarr/TMs/PFAF_${continent}${area}",
    "MERIT_TM": "${data_path}/${name}/zarr/TMs/MERIT_FLOWLINES_${continent}${area}",
    "streamflow": "${data_path}/streamflow/zarr/dpl_v2/${zone}",
    "q_prime": "${data_path}/${name}/zarr/q_prime/${zone}/"
    }
}'''

data_dict = json.loads(json_data)
cfg = OmegaConf.create(data_dict)

In [20]:
streamflow_prediction = zarr.open(Path(cfg.zarr.streamflow))
huc_to_merit_TM = zarr.open(Path(cfg.zarr.HUC_TM), mode='r')
merit_to_edge_TM = zarr.open(Path(cfg.zarr.MERIT_TM), mode='r')

In [21]:
print(streamflow_prediction.tree())


/
 ├── HUC10 (2437,) int64
 ├── streamflow (350617, 2437) float64
 └── time (350617,) int64


In [22]:
print(huc_to_merit_TM.tree())


/
 ├── COMID (23294,) int64
 ├── HUC10 (2437,) object
 └── TM (2437, 23294) float64


In [23]:
print(merit_to_edge_TM.tree())


/
 ├── COMID (23294,) int64
 ├── EDGEID (96353,) <U11
 └── TM (23294, 96353) float64


In [24]:
xr_streamflow_to_huc_TM = xr.open_zarr(Path(cfg.zarr.streamflow))
xr_huc_to_merit_TM = xr.open_zarr(Path(cfg.zarr.HUC_TM))
xr_merit_to_edge_TM = xr.open_zarr(Path(cfg.zarr.MERIT_TM))

In [25]:
print(xr_streamflow_to_huc_TM["streamflow"].dims)
print(xr_huc_to_merit_TM["TM"].dims)
print(xr_merit_to_edge_TM["TM"].dims)

('time', 'HUC10')
('HUC10', 'COMID')
('COMID', 'EDGEID')


In [26]:
streamflow_merit = xr.dot(xr_streamflow_to_huc_TM['streamflow'], xr_huc_to_merit_TM['TM'])
streamflow_edges = xr.dot(streamflow_merit, xr_merit_to_edge_TM['TM'])

/home/tkb5476/.conda/envs/marquette_env/lib/python3.11/site-packages/dask/array/einsumfuncs.py:234: PerformanceWarning: Increasing number of chunks by factor of 16
  result = blockwise(
/home/tkb5476/.conda/envs/marquette_env/lib/python3.11/site-packages/dask/array/einsumfuncs.py:234: PerformanceWarning: Increasing number of chunks by factor of 64
  result = blockwise(


In [41]:
streamflow_edges

<xarray.DataArray (time: 350617, EDGEID: 96353)>
dask.array<sum-aggregate, shape=(350617, 96353), dtype=float64, chunksize=(5479, 1506), chunktype=numpy.ndarray>
Coordinates:
  * time     (time) datetime64[ns] 1980-01-01 1980-01-01T01:00:00 ... 2019-12-31
  * EDGEID   (EDGEID) <U11 '73021958_0' '73022906_0' ... '73017442_2'

In [ ]:
output_path = Path(cfg.zarr.q_prime})
for col_idx in tqdm(range(streamflow_edges.shape[1]), desc="Processing Columns"):
    data = streamflow_edges[:, col_idx]
    data.to_netcdf( / f"{col_idx}.nc)